In [1]:
# https://colab.research.google.com/github/vsitzmann/siren/blob/master/explore_siren.ipynb

In [2]:
!pip install -U scikit-image
!pip install -U miplib

Requirement already up-to-date: scikit-image in ./miniconda3/envs/thesis/lib/python3.7/site-packages (0.18.1)
Requirement already up-to-date: miplib in ./miniconda3/envs/thesis/lib/python3.7/site-packages (1.0.6)


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os

from PIL import Image
from torchvision.transforms import Resize, Compose, ToTensor, Normalize, ToPILImage
import numpy as np
from matplotlib.ticker import FormatStrFormatter
import skimage
import matplotlib.pyplot as plt

import scipy

import time
import gc

def get_mgrid(sidelen, dim=2):
    '''Generates a flattened grid of (x,y,...) coordinates in a range of -1 to 1.
    sidelen: int
    dim: int'''
    tensors = tuple(dim * [torch.linspace(-1, 1, steps=sidelen)])
    mgrid = torch.stack(torch.meshgrid(*tensors), dim=-1)
    mgrid = mgrid.reshape(-1, dim)
    return mgrid

!pip install gemmi
import gemmi

In [4]:
class SineLayer(nn.Module):
    # See paper sec. 3.2, final paragraph, and supplement Sec. 1.5 for discussion of omega_0.
    
    # If is_first=True, omega_0 is a frequency factor which simply multiplies the activations before the 
    # nonlinearity. Different signals may require different omega_0 in the first layer - this is a 
    # hyperparameter.
    
    # If is_first=False, then the weights will be divided by omega_0 so as to keep the magnitude of 
    # activations constant, but boost gradients to the weight matrix (see supplement Sec. 1.5)
    
    def __init__(self, in_features, out_features, bias=True,
                 is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        self.init_weights()
    
    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 
                                             1 / self.in_features)      
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0, 
                                             np.sqrt(6 / self.in_features) / self.omega_0)
        
    def forward(self, input):
        return torch.sin(self.omega_0 * self.linear(input))
    
    def forward_with_intermediate(self, input): 
        # For visualization of activation distributions
        intermediate = self.omega_0 * self.linear(input)
        return torch.sin(intermediate), intermediate
    
    
class Siren(nn.Module):
    def __init__(self, in_features, hidden_features, hidden_layers, out_features, outermost_linear=False, 
                 first_omega_0=30, hidden_omega_0=30.):
        super().__init__()
        
        self.net = []
        self.net.append(SineLayer(in_features, hidden_features, 
                                  is_first=True, omega_0=first_omega_0))

        for i in range(hidden_layers):
            self.net.append(SineLayer(hidden_features, hidden_features, 
                                      is_first=False, omega_0=hidden_omega_0))

        if outermost_linear:
            final_linear = nn.Linear(hidden_features, out_features)
            
            with torch.no_grad():
                final_linear.weight.uniform_(-np.sqrt(6 / hidden_features) / hidden_omega_0, 
                                              np.sqrt(6 / hidden_features) / hidden_omega_0)
                
            self.net.append(final_linear)
        else:
            self.net.append(SineLayer(hidden_features, out_features, 
                                      is_first=False, omega_0=hidden_omega_0))
        
        self.net = nn.Sequential(*self.net)
    
    def forward(self, coords):
        coords = coords.clone().detach().requires_grad_(True) # allows to take derivative w.r.t. input
        output = self.net(coords)
        return output, coords        

    def forward_with_activations(self, coords, retain_grad=False):
        '''Returns not only model output, but also intermediate activations.
        Only used for visualizing activations later!'''
        activations = OrderedDict()

        activation_count = 0
        x = coords.clone().detach().requires_grad_(True)
        activations['input'] = x
        for i, layer in enumerate(self.net):
            if isinstance(layer, SineLayer):
                x, intermed = layer.forward_with_intermediate(x)
                
                if retain_grad:
                    x.retain_grad()
                    intermed.retain_grad()
                    
                activations['_'.join((str(layer.__class__), "%d" % activation_count))] = intermed
                activation_count += 1
            else: 
                x = layer(x)
                
                if retain_grad:
                    x.retain_grad()
                    
            activations['_'.join((str(layer.__class__), "%d" % activation_count))] = x
            activation_count += 1

        return activations

In [5]:
def laplace(y, x):
    grad = gradient(y, x)
    return divergence(grad, x)


def divergence(y, x):
    div = 0.
    for i in range(y.shape[-1]):
        div += torch.autograd.grad(y[..., i], x, torch.ones_like(y[..., i]), create_graph=True)[0][..., i:i+1]
    return div


def gradient(y, x, grad_outputs=None):
    if grad_outputs is None:
        grad_outputs = torch.ones_like(y)
    grad = torch.autograd.grad(y, [x], grad_outputs=grad_outputs, create_graph=True)[0]
    return grad

In [6]:
create_structure_from_scratch = False

if create_structure_from_scratch: 
    # Getting test T20S map
    if not os.path.exists('emd_6287.map.gz'):
        !wget ftp://ftp.ebi.ac.uk/pub/databases/emdb/structures/EMD-6287/map/emd_6287.map.gz
    if not os.path.exists('emd_6287.map'):
        !gunzip 'emd_6287.map.gz'
    map_path = 'emd_6287.map'

    # https://bioinformatics.stackexchange.com/questions/13079/ccp4-file-to-a-python-3-numpy-array-or-similar-workaround
    test_map = gemmi.read_ccp4_map(map_path)
    test_map.setup()  # optional

    raw_structure = np.array(test_map.grid, copy=False)
    plt.imshow(raw_structure[:, :, raw_structure.shape[0] // 2])
    plt.show()
    
    structure = np.expand_dims(raw_structure, axis=-1)
    structure = structure[50:-50, 50:-50, 50:-50, :]
    structure = scipy.ndimage.zoom(structure, (0.5, 0.5, 0.5, 1))
    
    with open('t20s_original_structure_reduced.npy', 'wb') as f:
        np.save(f, structure)
else:
    structure = np.load('t20s_original_structure_reduced.npy')

In [7]:
output_dims = structure.shape[0]

In [ ]:
class ImageFitting(Dataset):
    def __init__(self, img, sidelength):
        super().__init__()
        img = torch.from_numpy(img)
        # self.pixels = img.permute(1, 2, 0).view(-1, 1)
        self.pixels = img.reshape(-1, 1)
        self.coords = get_mgrid(sidelength, 3)

    def __len__(self):
        return len(self.pixels)

    def __getitem__(self, idx):
        
            
        return self.coords[idx], self.pixels[idx]
    
for hidden_features in [256, 512, 768]:
    for omega in [30, 50, 70]:

        # Clears memory
        #################
        torch.cuda.empty_cache()
        gc.collect()
        #################

        particle_slice = ImageFitting(structure, output_dims)
        dataloader = DataLoader(particle_slice, batch_size=256, pin_memory=True, num_workers=0)

        siren_model = Siren(in_features=3, out_features=1, hidden_features=hidden_features, 
                          hidden_layers=7, outermost_linear=True, first_omega_0=omega, hidden_omega_0=omega)
        siren_model.cuda()

        loss_vals = []

        total_steps = 250 # Since the whole image is our dataset, this just means 500 gradient descent steps.
        steps_til_summary = 10

        optim = torch.optim.Adam(lr=1e-6, params=siren_model.parameters())

        # model_input, ground_truth = next(iter(dataloader))
        # model_input, ground_truth = model_input.cuda(), ground_truth.cuda()

        for step in range(total_steps):

            running_loss = 0.0

            epoch_loss = 0.0
            for batch, data in enumerate(dataloader):
                model_input, ground_truth = data
                model_input, ground_truth = model_input.cuda(), ground_truth.cuda()

                optim.zero_grad()

                model_output, coords = siren_model(model_input)    
                loss = ((model_output - ground_truth)**2).mean()
                loss.backward()
                optim.step()

                epoch_loss += loss.item()
                '''
                running_loss += loss.item()
                # if not step % steps_til_summary:
                if batch % 400 == 399:
                    print("Epoch: %d, Batch: %d, Loss %0.9f" % (step + 1, batch + 1, running_loss / 400))
                    running_loss = 0.0
                    # img_grad = gradient(model_output, coords)
                    # img_laplacian = laplace(model_output, coords)
                '''

                '''
                    fig, axes = plt.subplots(1,1, figsize=(18,6))
                    axes.imshow(model_output.cpu().view(output_dims,output_dims).detach().numpy())
                    # axes[1].imshow(img_grad.norm(dim=-1).cpu().view(256,256).detach().numpy())
                    # axes[2].imshow(img_laplacian.cpu().view(256,256).detach().numpy())
                    plt.show()
                '''

            epoch_loss /= len(dataloader)
            print("Epoch: %d, Loss %0.9f" % (step + 1, epoch_loss))
            loss_vals.append(epoch_loss)
            epoch_loss = 0.0

        with open('t20s_siren_reconstruction_losses_hidden' + str(hidden_features) +'_omega' + str(omega) + '.npy', 'wb') as f:
            np.save(f, np.array(loss_vals))

        torch.save(siren_model, 't20s_siren_model_hidden' + str(hidden_features) +'_omega' + str(omega) + '.pt')

        '''
        # # Model class must be defined somewhere
        model = torch.load(PATH)
        model.eval()
        '''